# Graphen - Teil 1

Ein **Graph** besteht aus **Knoten** und **Kanten**. Er kann gerichtet oder ungerichtet sein. Die Kanten 
können **gewichtet** sein (Kosten).  

<img src='graphen_01.png'>
<img src='graphen_02.png'>


Mit Graphen können binäre Beziehungen zwischen Objekten modelliert werden. Die Objekte werden
durch die Knoten, die Beziehungen durch die Kanten modelliert.  

* Orte: Entfernungen, Dauer
* Personen: ist befreundet, verheiratet mit
* Ereignisse: a muss vor b geschehen

#### Begriffe

<img src='graphen_03.png'>
<img src='graphen_04.png'>

* x ist zu y adjazent, x und y sind Nachbarn
* der Grad von y ist 2
* a ist Vorgänger von b, b ist Nachfolger von a
* der Eingangsgrad von b ist 2, der Ausgangsgrad ist 1
* Ein **Weg** ist eine Folge von adjazenten Knoten
* Ein **Kreis** ist ein Weg, der zurück zum Startknoten führt.
 

### Implementation von Graphen

Eine Implementation sollte zwei Dinge zügig können:
* Die Frage klären, ob es eine Kante von Knoten a nach Knoten b gibt,
* Alle Nachbarn von a durchlaufen.




#### Implementation eines Graphen durch Adjazenzlisten

Jedem Knoten wird eine Liste seiner Nachbarn zugeordnet. Wir werden statt Listen *sets* oder *dictionaries* verwenden.

Ein ungewichteter Graph wird durch ein dictionary implementiert, mit den Knoten als key und der Nachbarmenge als value.


<img src='graphen_10.png'>

In [1]:
G = {'a':set('bc'),
     'b':set('d'), 
     'c':set('bd'),
     'd':set('b')}

In [2]:
# alle Nachbarn von a durchlaufen: 
for v in G['a']:
    print(v)

c
b


In [3]:
# gibt es Kante von a nach b ? 
'b' in G['a']

True

In [4]:
# alle Knoten von G durchlaufen:
for v in G:
    print(v)

a
b
c
d


Für einen ungerichteten Graphen implementieren wir Pfeile in beide Richtungen.


Ein **gewichteter Graph** wird durch ein dictionary implementiert, mit den Knoten als key und einem dictionary, in dem die Kosten zu den Nachbarn verzeichnet sind, als value.

<img src='graphen_11.png'>

In [8]:
G = {'a': {'b':8, 'c':2},          
     'b': {'d':4},               
     'c': {'b':1, 'd':6},          
     'd': {'b':2}}

In [9]:
# alle Nachbarn von a durchlaufen
for v in G['a']:
    print(v)

b
c


In [10]:
# gibt es Kante von a nach b ?
'b' in G['a']  

True

In [11]:
# die Kosten der Kante von a nach b:  
G['a']['b']     

8

In [38]:
# alle Knoten von G durchlaufen:   
for v in G:   
    print(v)

a
b
c
d


### Erreichbarkeit (dfs)

<img src='graphen_10.png'>

Gegeben ein Graph G und ein Startknoten s. Welche Knoten sind von s erreichbar?


In [2]:
G = {'a':set('bc'),
     'b':set('d'), 
     'c':set('bd'),
     'd':set('b')}

 
    Setze alle Knoten auf nicht besucht
    dfs(s)
    Gib alle Knoten aus, die besucht wurden
    
    def dfs(v): 
        merke v als besucht
        für alle Nachbarn w von v:
            wenn w nicht besucht:
                dfs(w)
     

In [17]:
visited =  {v : False for v in G}       
def dfs(v):  
    visited[v] = True
    for w in G[v]:
        if not visited[w]:
            dfs(w) 
            
s = 'a' 
dfs(s)
result = [v for v in G if visited[v]]            
print(*result)

a b c d


Die Funktion **dfs** realisiert eine rekursive Tiefensuche (depth first search). <br>
Laufzeit: Jeder Knoten wird höchstens einmal besucht $O(|V|)$, jeder Knoten prüft seine Nachbarn. Die Zahl der Nachbarn liegt in $O(|E|)$, also insgesamt $O(|V|) + O(|E|)$

### Den Graph mit Tiefensuche traversieren

Traversieren heißt: alle Knoten einmal besuchen

<img src='reihenfolge_01.png'>

In [10]:
G = {
'a': set('bde'),
'b': set('ae'),
'c': set('gi'),
'd': set('ah'),
'e': set('abf'),
'f': set('eh'),
'g': set('ci'),
'h': set('df'),
'i': set('cg'),
}

visited =  {v : False for v in G}       
def dfs(v):  
    print(v, end=' ')
    visited[v] = True
    for w in sorted(G[v]):   # für alphabetische Reihenfolge der Nachbarn
        if not visited[w]:
            dfs(w) 
            
for v in sorted(G):          # für alphabetische Reihenfolge der Knoten
    if not visited[v]:
        dfs(v)
           
 

a b e f h d c g i 

### Kürzester Weg in einem ungewichteten Graphen (bfs)

Die Länge eines Pfades in einem ungewichteten Graphen ist die Anzahl seiner Kanten. Die Entfernung zweier Knoten 
in einem ungewichteten Graphen ist die Länge des kürzesten Pfades zwischen ihnen.

<img src='graphen_24.png'>

Die Entfernung zwischen S und H ist 3.

Die **Breitensuche (breadth first search, bfs)** traversiert den Graphen in Schichten nach der Entfernung zum Ausgangsknoten.
Zu jedem Knoten merkt man sich seinen Vorgänger (prev), dadurch entsteht ein **shortest-path Baum**.
(Ein Baum ist ein zusammenhängender Graph, der keine Kreise enthält). Daraus lässt sich der kürzeste
Weg vom Ausgangsknoten zu jedem anderen Knoten rekonstruieren.

<img src='graphen_25.png'>

Als Datenstruktur für die Knoten, die wir noch untersuchen wollen, verwenden wir eine Schlange. Eine Schlange realisiert das FIFO-Prinzip (First-In, First-Out).

    # Breitensuche mit Ausgangsknoten s:
    Für jeden Knoten u in G: 
        dist(u) = unendlich 
        prev(u) = None
    
    Füge s in eine Schlange Q ein 
    dist(s) = 0
    
    Solange Q nicht leer:
        hole Knoten u aus der Schlange
            für alle Nachbarn v von u:
                falls dist(v) unendlich:
                    Füge v in die Schlange Q ein
                    dist(v) = dist(u) + 1
                    prev(v) = u

In [12]:
from collections import deque

G = {
'a': set('sb'),
'b': set('acgh'),
'c': set('bs'),
'd': set('sef'),
'e': set('sd'),
'f': set('dg'),
'g': set('bfh'),
'h': set('bg'),
's': set('acde')
}

inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

s = 's'         # Startknoten
dist[s] = 0
Q = deque([s])          
while Q:
    u = Q.popleft()
    #for v in G[u]:
    for v in sorted(G[u]):       #  für alphabetische Reihenfolge  
        if dist[v] == inf:
            Q.append(v)
            dist[v] = dist[u]+1
            prev[v] = u

print(prev)
print(dist)

{'a': 's', 'b': 'a', 'c': 's', 'd': 's', 'e': 's', 'f': 'd', 'g': 'b', 'h': 'b', 's': None}
{'a': 1, 'b': 2, 'c': 1, 'd': 1, 'e': 1, 'f': 2, 'g': 3, 'h': 3, 's': 0}


Das dictionary *prev* codiert den shortest-path Baum. Das dictionary *dist* enthält die kürzesten Entfernungen zum Startknoten. Einen Pfad erhalten wir, indem wir den Baum vom Ziel bist zum Startknoten rückwärts gehen.

In [13]:
def reconstructPath(s,u,prev):
    result = []
    while u != s:
        result.append(u)
        u = prev[u]
    result.append(s)
    result.reverse()
    return result
    
ziel = 'h'
print('Pfad von',s,'nach',ziel,':',*reconstructPath(s,ziel,prev))
print('Länge =',dist[ziel])

Pfad von s nach h : s a b h
Länge = 3


### Kürzester Weg: Der Algorithmus von Dijkstra

Der Algorithmus von Dijkstra findet  in einem gerichteten, mit nichtnegativen Kosten gewichteten
Graphen die kürzesten Wege von einem Startknoten zu allen anderen Knoten. Er löst das
**single source shortest paths**-Problem.

<img src='graphen_26.png'>

    # Algorithmus von Dijkstra
    Setze dist des Startknotens s vorläufig auf 0,
        alle anderen vorläufig auf unendlich. 
    Setze prev aller Knoten auf None
    Solange es noch vorläufige Knoten gibt
        setze dist des billigsten vorläufigen Knoten auf endgültig
        Für jeden Nachbarn v von u:   
            dist(v) = min(dist(v),dist(u) + Kosten von u nach v)   # Relaxieren
            prev(v) = letzter Knoten auf dem Weg zu den minimalen Kosten

In [1]:
G = {
'a': {'b':2, 'c':9},
'b': {'c':5, 'd':13},
'c': {'d':6, 'e':9},
'd': {'e':1, 'f':5},
'e': {'f':2},
'f': {}
}

from heapq import  heappop, heappush

def reconstructPath(s,u,prev):
    temp = []
    while u != s:
        temp.append(u)
        u = prev[u]
    temp.append(s)
    temp.reverse()
    return temp

inf = float('inf')
dist = {v:inf for v in G}
prev = {v:None for v in G}

s = 'a'         # startknoten
dist[s] = 0
endgueltig = set()    

vorlaeufig = [(dist[v],v) for v in G]

while vorlaeufig:
    _, u = heappop(vorlaeufig)
    if u in endgueltig: continue
    endgueltig.add(u)
    for v in G[u]:
        if dist[v] > dist[u] + G[u][v]:
            dist[v] = dist[u] + G[u][v]
            prev[v] = u
            heappush(vorlaeufig,(dist[v],v))
ziel = 'f'
print('Pfad von',s,'nach',ziel,':',*reconstructPath('a','f',prev))
print('Distanz:',dist[ziel])

Pfad von a nach f : a b c d e f
Distanz: 16


Während des Algorithmus müssen wir uns aus den vorläufig markierten Knoten laufend einen billigsten suchen. 
Dazu nutzen wir einen Heap. 
Die Laufzeit hängt ab vom Aufwand der Operationen zur Entnahme der Knoten aus dem Heap.
Bei einen Fibonacci-Heap ist dies $O(1)$ und der Aufwand also $O(\left|V\right|+\left|E\right|)$.
Aufwand bei einem Min-Heap: $O((\left|V\right|+\left|E\right|)\cdot\log\left|V\right|)$.